In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd
import numpy as np
import os

In [3]:
year = 2022
month = 2

# Define the output directory
output_dir = 'output'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the input file URL and output file path
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'{output_dir}/fhv_tripdata_{year:04d}-{month:02d}.parquet'

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data(input_file)

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
# Prepare a DataFrame with the ride_id and predicted_duration

df_result = pd.DataFrame()
# Create 'ride_id' column
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result['predicted_duration'] = y_pred

In [10]:
# Save the DataFrame to a parquet file
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [11]:
# Get the size of the output file
file_size = os.path.getsize(output_file)
file_size_mb = file_size / 1024 / 1024
print(file_size_mb)

57.2156286239624


In [12]:
!ls -lh output/

total 58M
-rw-rw-r-- 1 ubuntu ubuntu 58M Jun 20 10:20 fhv_tripdata_2022-02.parquet
